Criar uma coleção:

In [7]:
!pip install python-dotenv
!pip install langchain_huggingface
!pip install --upgrade pymilvus
!pip install PyPDF2
!pip install sentence_transformers
!pip install grpcio==1.60.0 

     |████████████████████████████████| 3.6 MB 4.9 MB/s eta 0:00:01
     |████████████████████████████████| 9.4 MB 60.6 MB/s eta 0:00:01
     |████████████████████████████████| 227 kB 31.8 MB/s eta 0:00:01
     |████████████████████████████████| 417 kB 45.6 MB/s eta 0:00:01
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.21.4
    Uninstalling huggingface-hub-0.21.4:
      Successfully uninstalled huggingface-hub-0.21.4
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 2.5.1
    Uninstalling sentence-transformers-2.5.1:
      Successfully uninstalled 

In [14]:
import os
import logging
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility

from langchain_huggingface import HuggingFaceEndpointEmbeddings
#from langchain_milvus import Milvus

import PyPDF2

### conexão ao milvus, criação de coleção e indexação de dados

In [15]:
#conexão com milvus
host            = 'watsonxdata'
port            = 19530
user            = 'ibmlhadmin'
password        = 'password'
server_pem_path = '/tmp/presto.crt'


In [16]:
from pymilvus import(
    Milvus,
    IndexType,
    Status,
    connections,
    FieldSchema,
    DataType,
    Collection,
    CollectionSchema,
)

connections.connect(alias='default',
                   host=host,
                   port=port,
                   user=user,
                   password=password,
                   server_pem_path=server_pem_path,
                   server_name='watsonxdata',
                   secure=True)

In [18]:
logger = logging.getLogger(__name__)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler = logging.StreamHandler()
handler.setLevel(logging.INFO)
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.info("Logger initialized")

In [19]:
#coleção compatível com LangChain
def create_milvus_collection(collection_name):
    if utility.has_collection(collection_name):
        utility.drop_collection(collection_name)
        logger.info(f"Dropped existing collection: {collection_name}")

    fields = [
        FieldSchema(name='pk', dtype=DataType.INT64, is_primary=True, auto_id=True),
        FieldSchema(name='text', dtype=DataType.VARCHAR, max_length=65_535),
        FieldSchema(name='vector', dtype=DataType.FLOAT_VECTOR, dim=384)
    ]
    
    schema = CollectionSchema(fields=fields, description="Coleção geral que atende as exigências do LangChain")
    collection = Collection(name=collection_name, schema=schema)
    
    index_params = {
        "metric_type": "L2",
        "index_type": "IVF_FLAT",
        "params": {"nlist": 2048}
    }
    collection.create_index(field_name="vector", index_params=index_params)
    logger.info(f"Created collection: {collection_name}")
    return collection

In [20]:
create_milvus_collection("Teste_ML")

<Collection>:
-------------
<name>: Teste_ML
<description>: General collection for id, text and vector
<schema>: {'auto_id': True, 'description': 'General collection for id, text and vector', 'fields': [{'name': 'pk', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': True}, {'name': 'text', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 65535}}, {'name': 'vector', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 384}}]}